In [59]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry 
from std_msgs.msg import Bool
import ipywidgets as widgets
from IPython.display import display
import assignment_2_2023.msg
from assignment_2_2023.msg import PlanningAction
from assignment_2_2023.msg import Info

In [ ]:
def on_feedback(action_feedback):
    global canCancel, send_coordinates
    
    # We can preempt the robot only if the state machine is in the "ACTIVE" status
    canCancel = (action_feedback.status.status == action_feedback.status.ACTIVE)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"

In [ ]:
def on_result(action_result):
    global canCancel, send_coordinates
    # We can preempt the robot only if the state machine is not in the "SUCCEEDED" or in the "PREEMPTED" status
    canCancel = not(action_result.status.status == action_result.status.SUCCEEDED or action_result.status.status == action_result.status.PREEMPTED)
    if(canCancel):
        send_coordinates.description = "Cancel"
    else:
        send_coordinates.description = "Send Coordinates"
    

In [8]:
def on_coordinates(msg):
    global pub_coordinates, info, x_coordinate, y_coordinate
    
    # rospy.loginfo("Velocity: %f, %f", vel_linear_x, vel_angular_z)
    
    # Prepare and publish the info message
    info = Info()
    info.x = x_coordinate
    info.y = y_coordinate
    # I do not actually need the velocity, but I use the message to publish the coordinates
    info.vel_linear_x = 0
    info.vel_angular_z = 0
    
    pub_coordinates.publish(info)

In [55]:
# def on_button_clicked(b):
#     global pub_cancel
#     pub_cancel.publish(True)
#     print("Button clicked.")

def on_coordinates_sent(b):
    global x_coordinate, y_coordinate, canCancel, pub_coordinates
    # if(info.x != x_coordinate.value or info.y != y_coordinate.value):
        # change the button to cancel name and function
    if(canCancel and b.description == "Cancel"):
        print("Cancelled")
        b.description = "Send Coordinates"
    elif(b.description == "Send Coordinates"):
        info = Info()
        info.x = x_coordinate
        info.y = y_coordinate
        # I do not actually need the velocity, but I use the message to publish the coordinates
        info.vel_linear_x = 0
        info.vel_angular_z = 0
        pub_coordinates.publish(info)
        print("Coordinates sent with x = %.1f, y = %.1f" % (x_coordinate.value, y_coordinate.value))
        print("Sent")
        b.description = "Cancel"

In [5]:
global pub_coordinates, pub_cancel
# create a publisher for the coordinates goal topic
pub_coordinates = rospy.Publisher('/goal/coordinates', Odometry, queue_size=1)

# create a publisher for the cancel goal topic
pub_cancel = rospy.Publisher('/goal/cancel', Bool, queue_size=1)

# Subscriber for feedback/status
sub_feedback = rospy.Subscriber("/reaching_goal/feedback", assignment_2_2023.msg.PlanningActionFeedback, on_feedback)
    
# Subscriber for result
sub_result = rospy.Subscriber("/reaching_goal/result", assignment_2_2023.msg.PlanningActionResult, on_result)


# button = widgets.Button(description="Click Me!") 
# output = widgets.Output()
# display(button, output)
# #handle changes, the observe method of the widget can be 

# button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

Button clicked.
Button clicked.
Button clicked.


In [58]:
global x_coordinate, y_coordinate, send_coordinates
x_coordinate = widgets.FloatSlider( 
value=7.0,
min=0.0, 
max=10.0, 
step=0.1,
description='X:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
y_coordinate = widgets.FloatSlider( 
value=7.0,
min=0.0, 
max=10.0, 
step=0.1,
description='Y:', 
disabled=False, 
continuous_update=False, 
orientation='horizontal', 
readout=True
)
display(x_coordinate, y_coordinate)

send_coordinates = widgets.Button(description="Send Coordinates")
output = widgets.Output()
display(send_coordinates, output)
send_coordinates.on_click(on_coordinates_sent)


FloatSlider(value=7.0, continuous_update=False, description='X:', max=10.0)

FloatSlider(value=7.0, continuous_update=False, description='Y:', max=10.0)

Button(description='Send Coordinates', style=ButtonStyle())

Output()

Coordinates sent with x = 7.0, y = 7.0
Sent
Cancelled
Coordinates sent with x = 7.0, y = 7.0
Sent
Cancelled
Coordinates sent with x = 7.0, y = 3.4
Sent
Cancelled
